# Clustering with Mean Shift

In [1]:
# Importing libraries and data
import pandas as pd

In [5]:
# Using titanic data set which is avaialbe at kaggle
titanic_data = pd.read_csv(r"C:\KD_Doc\ML work\Lab_Home_work\titanic_train.csv")
titanic_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


#### Using this data very differently - So i am doing grouping of the passangers wiht similar characteristics.
#### So we are droping those variable which is quite specific about passanger and focus on other variables

In [6]:
# droping unnecessary columns which are not required
titanic_data.drop(["PassengerId","Name","Ticket","Cabin"],axis=1,inplace=True)
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


# Preprocessing of the data

In [7]:
from sklearn import preprocessing

In [10]:
# Converting gender column into numeric form
le = preprocessing.LabelEncoder()
titanic_data["Sex"] = le.fit_transform(titanic_data["Sex"].astype(str))

In [11]:
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,S
1,1,1,0,38.0,1,0,71.2833,C
2,1,3,0,26.0,0,0,7.9250,S
3,1,1,0,35.0,1,0,53.1000,S
4,0,3,1,35.0,0,0,8.0500,S


In [13]:
# Creating dummie variable of "Embarked" column
titanic_data = pd.get_dummies(titanic_data,columns=["Embarked"])
titanic_data.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
0,0,3,1,22.0,1,0,7.2500,0,0,1
1,1,1,0,38.0,1,0,71.2833,1,0,0
2,1,3,0,26.0,0,0,7.9250,0,0,1
3,1,1,0,35.0,1,0,53.1000,0,0,1
4,0,3,1,35.0,0,0,8.0500,0,0,1


In [14]:
# Checking missing value or invailed rows available in dataset
titanic_data.isnull().sum()

Survived        0
Pclass          0
Sex             0
Age           177
SibSp           0
Parch           0
Fare            0
Embarked_C      0
Embarked_Q      0
Embarked_S      0
dtype: int64

In [15]:
titanic_data[titanic_data.isnull().any(axis =1)]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
5,0,3,1,NaN,0,0,8.4583,0,1,0
17,1,2,1,NaN,0,0,13.0000,0,0,1
19,1,3,0,NaN,0,0,7.2250,1,0,0
26,0,3,1,NaN,0,0,7.2250,1,0,0
28,1,3,0,NaN,0,0,7.8792,0,1,0
...,...,...,...,...,...,...,...,...,...,...
859,0,3,1,NaN,0,0,7.2292,1,0,0
863,0,3,0,NaN,8,2,69.5500,0,0,1
868,0,3,1,NaN,0,0,9.5000,0,0,1
878,0,3,1,NaN,0,0,7.8958,0,0,1


Both the codes shows age columns has Nan value, so we are droping those rows which has Nan value.

In [16]:
# droping NA value rows from data
titanic_data = titanic_data.dropna()

# Model building- Mean shift alog

In [17]:
from sklearn.cluster import MeanShift

In [18]:
# Initialization of object
analyzer = MeanShift(bandwidth=50)

In [19]:
# Fitting a model
analyzer.fit(titanic_data)

MeanShift(bandwidth=50, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

Overhear bandwidth is a hyperparameter and we don't need to guess, sklearn give a functionality which gives best bandwith as per our data. See below process to calculate best bandwidth as per data

#### Bandwidth function

In [21]:
from sklearn.cluster import estimate_bandwidth
estimate_bandwidth(titanic_data)

30.44675914497196

For our data best bandwidth is 30

In [22]:
# we can go ahead with bandwidth 50 for further process
labels = analyzer.labels_

In [23]:
import numpy as np

In [24]:
np.unique(labels)

array([0, 1, 2], dtype=int64)

With bandwidth of 50 we had 3 cluster and each and everypoint is associated with these 3 cluster only

In [25]:
# Adding a new cloumn - cluster group column
titanic_data["cluster_group"] = np.nan
data_length = len(titanic_data)
for i in range(data_length):
    titanic_data.iloc[i,titanic_data.columns.get_loc("cluster_group")] = labels[i]

C:\Users\Kuldeep.Singh5\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Kuldeep.Singh5\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [29]:
titanic_data["cluster_group"].value_counts()

0.0    680
1.0     31
2.0      3
Name: cluster_group, dtype: int64

In [30]:
# Grouping of data by cluster and checking summary of data
titanic_cluster_data = titanic_data.groupby(["cluster_group"]).mean()
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
cluster_group,,,,,,,,,,
0.0,0.388235,2.298529,0.651471,29.559191,0.505882,0.404412,25.415625,0.167647,0.041176,0.788235
1.0,0.741935,1.000000,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.000000,0.580645
2.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000


In [31]:
# Including count count column in cluster summary
titanic_cluster_data["Counts"] = pd.Series(titanic_data.groupby(["cluster_group"]).size())
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,0.388235,2.298529,0.651471,29.559191,0.505882,0.404412,25.415625,0.167647,0.041176,0.788235,680
1.0,0.741935,1.000000,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.000000,0.580645,31
2.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000,3


In [32]:
# Reading 2nd cluster where survival rate is 74 % which is high
titanic_data[titanic_data["cluster_group"]==1].describe()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
count,31.000000,31.0,31.000000,31.000000,31.000000,31.000000,31.000000,31.000000,31.0,31.000000,31.0
mean,0.741935,1.0,0.258065,32.223226,0.709677,1.032258,192.008732,0.419355,0.0,0.580645,1.0
std,0.444803,0.0,0.444803,15.327994,1.006431,1.016001,50.203716,0.501610,0.0,0.501610,0.0
min,0.000000,1.0,0.000000,0.920000,0.000000,0.000000,133.650000,0.000000,0.0,0.000000,1.0
25%,0.500000,1.0,0.000000,22.000000,0.000000,0.000000,151.550000,0.000000,0.0,0.000000,1.0
50%,1.000000,1.0,0.000000,31.000000,0.000000,1.000000,164.866700,0.000000,0.0,1.000000,1.0
75%,1.000000,1.0,0.500000,41.500000,1.000000,2.000000,237.522900,1.000000,0.0,1.000000,1.0
max,1.000000,1.0,1.000000,64.000000,3.000000,4.000000,263.000000,1.000000,0.0,1.000000,1.0


This particular group avg age is 32+ which  and ticket fare is 192 which high. 

In [34]:
# Details description of cluster 
titanic_data[titanic_data["cluster_group"]==1]

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,cluster_group
27,0,1,1,19.00,3,2,263.0000,0,0,1,1.0
88,1,1,0,23.00,3,2,263.0000,0,0,1,1.0
118,0,1,1,24.00,0,1,247.5208,1,0,0,1.0
195,1,1,0,58.00,0,0,146.5208,1,0,0,1.0
268,1,1,0,58.00,0,1,153.4625,0,0,1,1.0
269,1,1,0,35.00,0,0,135.6333,0,0,1,1.0
297,0,1,0,2.00,1,2,151.5500,0,0,1,1.0
299,1,1,0,50.00,0,1,247.5208,1,0,0,1.0
305,1,1,1,0.92,1,2,151.5500,0,0,1,1.0
311,1,1,0,18.00,2,2,262.3750,1,0,0,1.0


# Building model with bandwidth = 30

In [35]:
# Initialization of object
analyzer = MeanShift(bandwidth=30)

In [36]:
# Fitting a model
analyzer.fit(titanic_data)

MeanShift(bandwidth=30, bin_seeding=False, cluster_all=True, max_iter=300,
          min_bin_freq=1, n_jobs=None, seeds=None)

In [42]:
# we can go ahead with bandwidth 50 for further process
labels = analyzer.labels_

In [43]:
np.unique(labels)

array([0, 1, 2, 3, 4], dtype=int64)

In [44]:
# Adding a new cloumn - cluster group column
titanic_data["cluster_group"] = np.nan
data_length = len(titanic_data)
for i in range(data_length):
    titanic_data.iloc[i,titanic_data.columns.get_loc("cluster_group")] = labels[i]

In [45]:
titanic_data["cluster_group"].value_counts()

0.0    558
1.0    108
2.0     30
3.0     15
4.0      3
Name: cluster_group, dtype: int64

In [46]:
# Grouping of data by cluster and checking summary of data
titanic_cluster_data = titanic_data.groupby(["cluster_group"]).mean()
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S
cluster_group,,,,,,,,,,
0.0,0.336918,2.525090,0.679211,28.256720,0.439068,0.370968,15.434139,0.121864,0.046595,0.831541
1.0,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,0.333333,0.018519,0.629630
2.0,0.733333,1.000000,0.366667,32.430667,0.600000,0.866667,131.183883,0.500000,0.000000,0.500000
3.0,0.733333,1.000000,0.266667,30.333333,1.000000,1.333333,239.991940,0.533333,0.000000,0.466667
4.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000


In [47]:
# Including count count column in cluster summary
titanic_cluster_data["Counts"] = pd.Series(titanic_data.groupby(["cluster_group"]).size())
titanic_cluster_data

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Counts
cluster_group,,,,,,,,,,,
0.0,0.336918,2.525090,0.679211,28.256720,0.439068,0.370968,15.434139,0.121864,0.046595,0.831541,558
1.0,0.611111,1.296296,0.527778,36.148148,0.814815,0.500000,65.622688,0.333333,0.018519,0.629630,108
2.0,0.733333,1.000000,0.366667,32.430667,0.600000,0.866667,131.183883,0.500000,0.000000,0.500000,30
3.0,0.733333,1.000000,0.266667,30.333333,1.000000,1.333333,239.991940,0.533333,0.000000,0.466667,15
4.0,1.000000,1.000000,0.666667,35.333333,0.000000,0.333333,512.329200,1.000000,0.000000,0.000000,3


Summary - 
When we change bandwidth from 50 to 30, so number of cluster also change and gives us better bifurcation of all ther variable
gives better pattern about passanger, how this parttern effect survival rate of passanger's.